Tài liệu này mang giấy phép Creative Commons Attribution (CC BY). (c) Nguyễn Ngọc Sáng, Zhukovsky 06/2019.

[@SangVn](https://github.com/SangVn) [@VnCFD](https://vncfdgroup.wordpress.com/)

*Thực hành CFD với Python!*

# Bài 23. Thiết lập module setup, run. Tính bài toán dòng chảy trên âm qua dốc 

Để giải quyết bài toán dòng chảy trên âm qua dốc, cần thiết lập các thông số cần thiết trong module `setup.py` và gọi các hàm tính trong `run.py`.

**1. Module setup:**
- Hàm thiết lập các thông số trường ban đầu
- Hàm thiết lập điều kiện biên
- Các thông số khác: CFL, time_target, mesh_filename, field_filename...

In [2]:
# coding: utf-8
# Module setup.py

from numpy import array
from fluxes import flux_roe
from boco import boco_dict, set_freestream
from functions import P2U, rho

P_inflow = array([rho(293.15, 101325.0), 660.0, 0.0, 101325.0]) # freestream

# cho thông số ban đầu bằng thông số dòng chảy vào 
def init_field(cells):
    P_t0 = P_inflow
    U_t0 = P2U(P_t0)
    for cell in cells[:]:
        cell.P = P_t0.copy()
        cell.U = U_t0.copy()
    
# điều kiện biên
def set_boco(sides):
    set_freestream(P_inflow)
    boco_name = ['supersonic_inflow', 'supersonic_outflow', 'wall', 'wall']
    boco_func = [boco_dict[name] for name in boco_name]
    sides.set_boco(boco_func)

# các thông số khác
CFL = 0.85
time_target = None
iter_target = 1500

write_field_iter = 1000 # thời điểm ghi kết quả giữa chừng
flux_func = flux_roe

mesh_filename = 'data/mach_2_mesh.dat'
field_filename = 'data/mach_2_cell_data.dat'

**2. Module run:**
- Nhập lưới: xác định kích thước Nj, Ni và tạo độ điểm lưới
- Tạo lớp dữ liệu cells, sides
- Thiết lập, lưu trường ban đầu. **Trong trường hợp ta muốn lấy kết quả đã có để tiếp tục tính toán thì không cần thực hiện bước này**
- Đọc trường ban đầu
- Gọi hàm solver để giải quyết bài toán
- Biểu diễn kết quả 

In [ ]:
# coding: utf-8
from data import Cells, Sides
from setup import init_field, mesh_filename, field_filename
from solver import eu_solver
from functions import import_mesh, read_field, write_cell_data, write_block_data, show_field, show_point_data

def run():
    # nhập lưới
    Nj, Ni, points = import_mesh(mesh_filename)

    # tạo dữ liệu cells, sides
    cells = Cells(Nj-1, Ni-1, points)
    sides = Sides(points, cells)

    # thiết lập trường ban đầu
    iter, time = 0, 0.0
    init_field(cells)
    write_cell_data(cells, iter, time, field_filename)

    # đọc trường ban đầu
    iter, time = read_field(cells, field_filename)

    # gọi eu_solver
    eu_solver(cells, sides, iter, time)
    
    # lưu kết quả ở dạng block_data 
    write_block_data(cells, points, iter, time, field_filename.replace('cell_data.dat', 'block_data.dat'))

    # biểu diễn kết quả, lưu ảnh
    show_field(cells, points, porc=1)

if __name__ == '__main__':
    run()

Như vậy ta có các module: **mesh.py, data.py, fluxes.py, solver.py, setup.py, run.py**. Thay đổi thời gian hoặc số vòng lặp trong `setup.py`, cho chạy module `run.py` thu được các kết quả như sau: 
<img src='img/Bai_23_1.gif'>
<img src='img/Bai_23_2.png' width = 600>

Về cách tạo file .gif hay clip hãy xem lại phần 1, [bài 9](https://nbviewer.jupyter.org/github/SangVn/CFD_Notebook_P1/blob/master/Bai_9.ipynb)

Bây giờ, để giải một bài toán khác chúng ta cần thay đổi những gì? Ở bài sau ta sẽ bàn luận về vấn đề này.
## [Bài 24. Bài toán dòng chảy quanh hình trụ, NACA0012, Crew Dragon. Điều kiện biên `joint, farfield`](Bai_24.ipynb)